In [1]:
import numpy as np
import math
import sys
from random import randint
from random import getrandbits,uniform,random
from matplotlib import pyplot as plt

In [2]:
class Geaty:
    def __init__(self, target_func=lambda x: x + 10*math.sin(5*x) + 7*math.cos(4*x), 
                 interval=[-5, 5], deciml_digits=6, chrmsome_size=50,cross_prob=1,mutate_prob=1
                 ):
        self.chrmsome_size = chrmsome_size  # 染色体条数
        self.target_func = target_func  # 要求解的目标函数
        self.interval = interval  # 取值区间
        self.deciml_digits = deciml_digits  # 要求精确到的小数点后位数
        
        self.chrmsome_code_length = self.determ_code_length()  # 确定编码长度（这里采用2进制编码）
        self.cross_prob = cross_prob  # 交叉概率，即能产生新后代的概率
        self.mutate_prob = mutate_prob
        
        self.individual_list = [self.encode() for _ in range(self.chrmsome_size)]  # 种群染色体集合
        self.individual_fitness_list = self.evaluate_fitness(self.individual_list)  # 计算种群中所有个体初始的适应度

        self.new_individual_list = []  # 新一代染色体集合
        self.new_individual_fitness_list = []  # 新一代染色体适应度集合
        
        self.best_chrmsome = self.individual_list[np.array(self.individual_fitness_list).argmax()]  # 最优染色体
        
        self.refer_solutn = self.plot_val()  # 参考解
          
    def plot_val(self, deciml_digits=3):  # 绘制图片并且返回真实解
        lin_space = 1/pow(10, deciml_digits)
        num_space = round((self.interval[1] - self.interval[0])/lin_space)
        x = [round(self.interval[0] + i*lin_space, deciml_digits) for i in range(num_space+1)]
        y = [self.target_func(argment) for argment in x]
        plt.plot(x, y)
        plt.show()
        arr_y = np.array(y)
        return x[arr_y.argmax()]
    
    def determ_code_length(self):
        '''
        :return:通过小数点位数和解区间计算每条染色体的编码长度(2进制编码) 
        '''
        sub_interval = (self.interval[1] - self.interval[0])*pow(10, self.deciml_digits)
        for c_length in range(sys.maxsize):
            if pow(2, c_length-1) <= sub_interval < pow(2, c_length):
                return c_length
    
    def encode(self):  # 编码过程
        while True:
            res = getrandbits(self.chrmsome_code_length)
            if self.interval[0] <= self.decode(res) <= self.interval[1]:
                return res
    
    def decode(self, chromosome):  # 解码过程
        return round(self.interval[0] + chromosome*(self.interval[1] - self.interval[0])/(pow(2, self.chrmsome_code_length)-1),self.deciml_digits)
    
    def cal_func(self, chromosome):  # 计算函数对应值
        val = self.decode(chromosome)
        return self.target_func(val)
    
    def evaluate_fitness(self, chrmsome_list):  # 评估所有染色体个体的适应度
        val_list = np.array([self.cal_func(each_chrmsome) for each_chrmsome in chrmsome_list])
        return val_list
    
    def roulette_selection(self, fitness_list):  # 轮盘对赌选择
        val_list += np.abs(val_list.min())
        valmin = val_list.min(axis=0)
        valmax = val_list.max(axis=0)
        
        sumFits = np.array(fitness_list).sum()
        rndPoint = uniform(0, sumFits)
        accumulator = 0.0
        for ind, val in enumerate(fitness_list):
            accumulator += val
            if accumulator >= rndPoint:
                return ind
    
    def cross(self, chrom1, chrom2):
        p = random()  # 随机概率
        n = 2 ** self.chrmsome_code_length - 1
        if chrom1 != chrom2 and p < self.cross_prob:
            t = randint(1, self.chrmsome_code_length - 1)  # 随机选择一点（单点交叉）
            mask = n << t  # << 左移运算符
            (l1, l2) = (chrom1 & mask, chrom2 & mask)  # & 按位与运算符：参与运算的两个值,如果两个相应位都为1,则该位的结果为1,否则为0
            mask = n >> (self.chrmsome_code_length - t)
            (r1, r2) = (chrom1 & mask, chrom2 & mask)
            (chrom1, chrom2) = (r1 + l2, r2 + l1)
        return chrom1, chrom2
    
    # 变异
    def mutate(self, chrom):
        p = random()
        if p < self.mutate_prob:
            t = randint(1, self.chrmsome_code_length)
            mask1 = 1 << (t - 1)
            mask2 = chrom & mask1
            if mask2 > 0:
                chrom = chrom & (~mask2)  # ~ 按位取反运算符：对数据的每个二进制位取反,即把1变为0,把0变为1
            else:
                chrom = chrom ^ mask1  # ^ 按位异或运算符：当两对应的二进位相异时，结果为1
        return chrom
    
    def refresh_best_chromosome(self):  # 更新最优染色体
        self.best_chrmsome = self.individual_list[np.array(self.individual_fitness_list).argmax()]
    
    def evove(self):  # 进行一次进化过程（假设有m个初始个体集合，每次取出两个进行杂交变异产生新种群，循环m/2次就认为完成了一次生产，然后按适应度进行优胜劣汰后认为完成了一次进化）
        pos_generate = 0
        while True:
            # 选择两个个体
            chrosome_p1 = self.roulette_selection(self.individual_fitness_list)
            chrosome_p2 = self.roulette_selection(self.individual_fitness_list)
            # 进行杂交
            new_individual_p1,new_individual_p2 = self.cross(chrosome_p1,chrosome_p2)
            # 变异
            new_individual_p1 = self.mutate(new_individual_p1)
            new_individual_p2 = self.mutate(new_individual_p2)
            # 存放在新个体集合中
            self.new_individual_list.extend([new_individual_p1, new_individual_p2])
            if pos_generate > self.chrmsome_size/2:
                break
            pos_generate += 2
        # 更新全局最优个体
        self.refresh_best_chromosome()
        # 处理新产生的个体
        
        pass
    
    def run(self):  # 完成算法运行的所有进化过程，遗传算法的运行引擎
        pass

In [3]:
geat_algorythm = Geaty(interval=[-1,1],deciml_digits=3)
res = geat_algorythm.individual_fitness_list